In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [2]:
train_df = pd.read_csv("./Data/resources.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541272 entries, 0 to 1541271
Data columns (total 4 columns):
id             1541272 non-null object
description    1540980 non-null object
quantity       1541272 non-null int64
price          1541272 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 47.0+ MB


In [3]:
total_price = pd.Series(train_df['quantity'].fillna(0.0).astype(int) * train_df['price'].fillna(0.0).astype(float))
total_price = total_price.rename('total_price', axis='columns')
total_price.head(10)

0    149.00
1     44.85
2      8.45
3     27.18
4     74.85
5     16.99
6      9.95
7     20.22
8     19.90
9     18.04
Name: total_price, dtype: float64

In [4]:
train_df = pd.concat([train_df, total_price], axis=1)
train_df.head(10)

,id,description,quantity,price,total_price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95,44.85
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59,27.18
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95,74.85
5,p069063,Last to Finish: A Story About the Smartest Boy...,1,16.99,16.99
6,p069063,"Mrs. Gorski, I Think I Have the Wiggle Fidgets...",1,9.95,9.95
7,p069063,"See-N-Read 1503905CQ Reading Tool - Book Size,...",2,10.11,20.22
8,p096795,"Brewster WPD90218 Wall Pops Flirt Dot, Set of ...",2,9.95,19.90
9,p096795,Brewster Wall Pops WPE99065 Peel & Stick Calyp...,2,9.02,18.04


In [6]:
train_df['description'] = train_df['description'].replace('\n','', regex=True).replace(',','').replace('"','')
#apply(lambda d: d.str.strip())

In [7]:
grouped_by_id = train_df.groupby('id')
grouped_by_id.size().head(10)

id
p000001     4
p000002    14
p000003     4
p000004    95
p000005     4
p000006     5
p000007     1
p000008    80
p000009    38
p000010     1
dtype: int64

In [8]:
import functools

def f_reduce_sum(list):
    val = functools.reduce(lambda x, y: x + y, list)
    return round(val, 2)

def f_reduce_concat(list):
    return functools.reduce(lambda x, y: str(x) + ' ' 
                                       + str(y)
                           , list)

In [9]:
train_df_grp1 = grouped_by_id.count()
train_df_grp1.head(10)

,description,quantity,price,total_price
id,,,,
p000001,4,4,4,4
p000002,14,14,14,14
p000003,4,4,4,4
p000004,95,95,95,95
p000005,4,4,4,4
p000006,5,5,5,5
p000007,1,1,1,1
p000008,80,80,80,80
p000009,38,38,38,38


In [10]:
train_df_grp2 = grouped_by_id.agg({
            'description': f_reduce_concat,
            'quantity': f_reduce_sum,
            'total_price': f_reduce_sum
            })
train_df_grp2.head(10)

,description,quantity,total_price
id,,,
p000001,"Cap Barbell 300 Pound Olympic Set, Grey Cap Ba...",7,833.63
p000002,10 Sony Headphones (BUY 9 GET 1 FREE) Belkin 6...,21,630.28
p000003,EE820X - Phonemic Awareness Instant Learning C...,4,298.97
p000004,A Bad Case of the Giggles Poems That Will Make...,98,1126.22
p000005,"Fitbit Zip Wireless Activity Tracker, Lime Fit...",8,702.31
p000006,Danny and the Dinosaur: 50th Anniversary Editi...,5,130.62
p000007,HP Chromebook 11 G4 11.6 Inch Laptop (Intel N2...,6,947.88
p000008,A Whale of a Tale Alvin and the Chipmunks Alvi...,80,296.99
p000009,Ball Don't Lie Baseball Card Adventure (4 Bk S...,45,335.40


In [11]:
train_df_grp2['cnt_by_id'] = train_df_grp1['quantity']

In [12]:
train_df_grp2 = train_df_grp2.rename({'quantity':'total_quantity'}, axis='columns')
train_df_grp2

,description,total_quantity,total_price,cnt_by_id
id,,,,
p000001,"Cap Barbell 300 Pound Olympic Set, Grey Cap Ba...",7,833.63,4
p000002,10 Sony Headphones (BUY 9 GET 1 FREE) Belkin 6...,21,630.28,14
p000003,EE820X - Phonemic Awareness Instant Learning C...,4,298.97,4
p000004,A Bad Case of the Giggles Poems That Will Make...,98,1126.22,95
p000005,"Fitbit Zip Wireless Activity Tracker, Lime Fit...",8,702.31,4
p000006,Danny and the Dinosaur: 50th Anniversary Editi...,5,130.62,5
p000007,HP Chromebook 11 G4 11.6 Inch Laptop (Intel N2...,6,947.88,1
p000008,A Whale of a Tale Alvin and the Chipmunks Alvi...,80,296.99,80
p000009,Ball Don't Lie Baseball Card Adventure (4 Bk S...,45,335.40,38


In [13]:
train_df_grp2.head(10)

,description,total_quantity,total_price,cnt_by_id
id,,,,
p000001,"Cap Barbell 300 Pound Olympic Set, Grey Cap Ba...",7,833.63,4
p000002,10 Sony Headphones (BUY 9 GET 1 FREE) Belkin 6...,21,630.28,14
p000003,EE820X - Phonemic Awareness Instant Learning C...,4,298.97,4
p000004,A Bad Case of the Giggles Poems That Will Make...,98,1126.22,95
p000005,"Fitbit Zip Wireless Activity Tracker, Lime Fit...",8,702.31,4
p000006,Danny and the Dinosaur: 50th Anniversary Editi...,5,130.62,5
p000007,HP Chromebook 11 G4 11.6 Inch Laptop (Intel N2...,6,947.88,1
p000008,A Whale of a Tale Alvin and the Chipmunks Alvi...,80,296.99,80
p000009,Ball Don't Lie Baseball Card Adventure (4 Bk S...,45,335.40,38


In [14]:
import csv
train_df_grp2.to_csv("B01.csv", index=True, quoting=csv.QUOTE_NONNUMERIC , quotechar='"')

In [ ]:
#dummy_df = pd.read_csv("B01_dummy.csv")
#dummy_df.info()